<a href="https://colab.research.google.com/github/alexledd/Blender-Colaboratory/blob/main/Simple_Blender_Render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt_text](https://repository-images.githubusercontent.com/656952604/b1ae251b-033f-48d7-a355-4cb880936ef2)

# **Render Blender file using Colaboratory**
* Only **Blender 3.X.X** are available in the dictionary. 📚
* Add your own version of Blender from the [Blender Repository](https://ftp.nluug.nl/pub/graphics/blender/release). 👷

## **🗒️ Notes:**
* Open `Files` tab on the left side of the web. 📂
* To copy folder or file's path, click on three dots on the right side of the file/folder inside the `Files` tab, `Copy path`. 🗄️
* Colab did not recognize file with space inside the filename, I recommend you to rename your file. *ex.* `"My Scene.blend"` to `"My_Scene.blend"`. 📝
* If you upload your .blend file directly from Colab, wait until it finishes uploading before running the first cell. 🔽
* If you encounter any issues or have suggestions, please let me know on **GitHub**. Thank you. 🤗



## **🎛️ Parameters:**
---

**🟠 Blender Options**
* 👷🛠️ `blender_version`: Blender version you are working with. Opening an older version of the blend file in a newer version is supported.
* 🗃️✍️ `project_name`: Your project name for better organization in the files tab.
* 📂🚶 `blend_file_path`: Path to your .blend file.
* 🔺📬 `gdrive_output` : All rendered will be save directly to your mounted Google Drive; Run `Mount Google Drive`
* 🗂️🔺 `organize_gdrive` : Move the original blend file to the auto-created project folder

---

**🟢 Rendering Options:**
* 🎞️📼 `animation`: Specify whether you want to render an animation or a still image. If using a still image, provide the frame number in `start_frame`.
* 1️⃣ `start_frame`, 🔟 `end_frame`: Specify the start and end frames for an animation. You can set both values to the same frame number (e.g., 0) to use the default frame in the blend file.

---

**🔵 GPU & CPU Rendering:**
* 🎬🖥️ `gpu_enabled`: Render using GPU. Run the `"Enable GPU Rendering"` cell.
* 🖥️🖱️ `cpu_enabled`: May slightly boost render performance.

**💿 OptiX Rendering:**
* 💽✅ `optix_enabled`: Enable OptiX Rendering Technology, which is faster than CUDA. Note that compatibility depends on the version of Blender, the project, and the allocated GPU.



In [ ]:
#@title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title # **Forms**

#@markdown
#@markdown ## **Blender Options**
blender_version = '3.5.1' #@param ['3.0.1','3.1.2','3.2.2' ,'3.3.8','3.4.1' , '3.5.1'] {allow-input: false}
project_name = 'Project_Name' #@param {type: 'string'}
blend_file_path = '/content/Your_Blender_File.blend' #@param {type: 'string'}
gdrive_output = False #@param {type:"boolean"}
organize_gdrive = False #@param {type:"boolean"}
#@markdown ---
#@markdown ## **Frame Options**
animation = False #@param {type: 'boolean'}
start_frame =  0#@param {type: 'integer'}
end_frame =  100#@param {type: 'integer'}
#@markdown ---
#@markdown ## **Rendering Options**
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
#@markdown

import os
from google.colab import drive

!mkdir /content/$project_name 2>&1 | awk '/exists/{print "Project folder already exists. Go continue your work :)"}'
!mkdir /content/$project_name/rendered 2>/dev/null
!cp $blend_file_path /content/$project_name/blend 2>/dev/null
!rm -r /content/sample_data 2>/dev/null

blend_file = os.path.basename(blend_file_path)

if gdrive_output or organize_gdrive:
    if os.path.exists('/content/drive'):
      pass
    else:
      drive.mount('/content/drive', force_remount=True)

if gdrive_output:
  !mkdir /content/drive/MyDrive/BlenderColab 2>&1 | awk '/exists/{print "BlenderColab folder already exists. Go continue your work :)"}'
  !mkdir /content/drive/MyDrive/BlenderColab/$project_name 2>/dev/null
  !mkdir /content/drive/MyDrive/BlenderColab/$project_name/rendered 2>/dev/null

if organize_gdrive:
    if "/content/drive/MyDrive" in blend_file_path:
        !mkdir /content/drive/MyDrive/BlenderColab/$project_name/blend 2>/dev/null
        !mv $blend_file_path /content/drive/MyDrive/BlenderColab/$project_name/blend 2>/dev/null
    else:
        valid_input = False
        while not valid_input:
            user_input = input("Do you want to move the blend file into your Google Drive? (Y/N): ")
            if user_input.lower() == "y":
                !mv $blend_file_path /content/drive/MyDrive/BlenderColab/$project_name/blend
                print("The blend file has been moved to your Google Drive to be organized :)")
                valid_input = True
            elif user_input.lower() == "n":
                valid_input = True
                # Do nothing
            else:
                print("Invalid input. Please enter either Y or N.")


In [ ]:
#@title NVIDIA GPU Check
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

In [ ]:
#@title Blender
# Add your own version of blender from https://ftp.nluug.nl/pub/graphics/blender/release
blender_url_dict = {'3.0.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.0/blender-3.0.1-linux-x64.tar.xz",
                    '3.1.2'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.1/blender-3.1.2-linux-x64.tar.xz",
                    '3.2.2'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.2/blender-3.2.2-linux-x64.tar.xz",
                    '3.3.8'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.3/blender-3.3.8-linux-x64.tar.xz",
                    '3.4.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz",
                    '3.5.1'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.5/blender-3.5.1-linux-x64.tar.xz"}

import os

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1
!rm /content/blender-$blender_version-linux-x64.tar.xz

In [ ]:
#@title Enable GPU rendering (add here for Custom Properties)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, set OptiX to False")
    renderer = "OPTIX"

if gpu_enabled:
  !cp /content/setgpu.py /content/$blender_version/

In [ ]:
#@title Render
%cd /content/$blender_version

if gdrive_output:
  output_path = f'/content/drive/MyDrive/BlenderColab/{project_name}/rendered/{project_name}-###'
else:
  output_path = f'/content/{project_name}/rendered/{project_name}-###'

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/$project_name/blend/{blend_file}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"